In [1]:
import os
import argparse
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset,DataLoader

import torchvision.utils as utils
import torchvision.transforms as transforms
import torchvision.models as models
import cv2

import copy
import glob
import torch.utils.data as udata
import h5py
import cv2
import random
import pywt
import matplotlib.pyplot as plt
from PIL import Image
import math
from skimage.measure.simple_metrics import compare_psnr
import scipy.io as sio
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False


In [2]:
#cpu/GPU训练
use_cuda = torch.cuda.is_available()
if use_cuda:
    device = "cuda:0" 
    dtype = torch.cuda.FloatTensor 
    imsize = 128 
else:
    device = "cpu"
    dtype = torch.FloatTensor
    # desired size of the output image
    imsize = 128  # use small size if no gpu
print(use_cuda)
print(device)

True
cuda:0


In [3]:
# desired size of the output image 数据预处理
loader = transforms.Compose([
    transforms.Resize([imsize, imsize]),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name) 
    image = Variable(loader(image)) #1,128,128 +grad+ grad_fn
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0) #
    return image

unloader = transforms.ToPILImage()  # reconvert into PIL image 
def imshow(tensor, title=None):
    image = tensor.clone().to(device) # we clone the tensor to not do changes on it
    image = image.view(3, imsize, imsize)  # remove the fake batch dimension
    image = unloader(image) #(W,H)
    plt.imshow(image) #显示图片
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated 动态更新图像，感觉没啥意义？

In [4]:
# MSE 内容损失，采用VGG网络conv4_1层输出的均方差损失 ，内容图为西电图标 自定义损失
class ContentLoss(nn.Module):

    def __init__(self, target, weight):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        self.target = target.detach() * weight 
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.weight = weight
        self.criterion = nn.MSELoss() #损失函数

    def forward(self, input): #前向传播 计算梯度
        self.loss = self.criterion(input * self.weight, self.target) #输入和目标都*权重，然后计算均方差损失
        self.output = input #返回不变？
        return self.output

    def backward(self, retain_graph=True):  #反向传播 最小化损失
        self.loss.backward(retain_graph=retain_graph) #保存中间变量，不然会直接在优化完这个损失之后清空变量
        return self.loss #返回损失

In [5]:
# GM 矩阵的计算  用于计算风格损失
class GramMatrix(nn.Module):

    def forward(self, input):
        a, b, c, d = input.size()  # a=batch size(=1)
        # b=number of feature maps chnnel number (h*w)
        # (c,d)=dimensions of a f. map (N=c*d)

        features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL (chnnel,h*w)

        G = torch.mm(features, features.t())  # compute the gram product 对特征向量以及他的转置矩阵做内积(点乘)，就是grim matrix

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(a * b * c * d) 

In [6]:
class StyleLoss(nn.Module): #f风格损失

    def __init__(self, target, weight):
        super(StyleLoss, self).__init__()
        self.target = target.detach() * weight
        self.weight = weight
        self.gram = GramMatrix()
        self.criterion = nn.MSELoss()

    def forward(self, input): #前向传播参数
        self.output = input.clone()
        self.G = self.gram(input)
        self.G.mul_(self.weight)
        self.loss = self.criterion(self.G, self.target)
        return self.output

    def backward(self, retain_graph=True): #反向传播损失
        self.loss.backward(retain_graph=retain_graph)
        return self.loss

In [7]:
cnn = models.vgg19(pretrained=True).features
# move it to the GPU if possible:
if use_cuda:
    cnn = cnn.to(device)#使用VGG网络模型

In [8]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4'] #内容特征层
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4']
#style_layers_default = [ 'conv_2', 'conv_4', 'conv_5','conv_6',]
#style_layers_default = ['conv_1', 'conv_3', 'conv_5', 'conv_9', 'conv_13'] #风格特征层
def get_style_model_and_losses(cnn, style_img, content_img,
                               style_weight=1000, content_weight=1,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    model = nn.Sequential()  # the new Sequential module network 初始化构建一个网络
    gram = GramMatrix()  # we need a gram module in order to compute style targets
    print('gram', gram)
    # move these modules to the GPU if possible:
    if use_cuda:
        model = model.cuda()
        gram = gram.cuda()

    i = 1
    for layer in list(cnn):
        if isinstance(layer, nn.Conv2d):
            name = "conv_" + str(i)
            model.add_module(name, layer) #我们之前只初始化了损失函数，现在，我们在网络模型中加入一个个网路层，现在遍历VGG，把他的层都加入
            #我的模型里面
            #print('新加入一层，名字为{}'.format(name))

            if name in content_layers:
                # add content loss:
                target = model(content_img).clone()
                content_loss = ContentLoss(target, content_weight)
                model.add_module("content_loss_" + str(i), content_loss)
                content_losses.append(content_loss)
                #print('我把这一层作为内容层{}'.format(name))

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).clone()
                # style_img经过提取后的特征展开为向量，然后构建gram矩阵
                target_feature_gram = gram(target_feature)
                style_loss = StyleLoss(target_feature_gram, style_weight)
                model.add_module("style_loss_" + str(i), style_loss)
                style_losses.append(style_loss)
                #print('我把这一层作为风格层{}'.format(name))

        if isinstance(layer, nn.ReLU):
            name = "relu_" + str(i)
            model.add_module(name, layer)

            i += 1

        if isinstance(layer, nn.MaxPool2d):
            name = "pool_" + str(i)
            model.add_module(name, layer)  # ***

    return model, style_losses, content_losses 

In [9]:
def get_input_param_optimizer(input_img): #优化器
    # this line to show that input is a parameter that requires a gradient
    input_param = nn.Parameter(input_img.data)
    optimizer = optim.LBFGS([input_param])  #目前最好的优化方法 低存储BFGS
    return input_param, optimizer

In [10]:
def run_style_transfer(cnn, content_img, style_img, input_img, num_steps=300,
                       style_weight=500, content_weight=1):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        style_img, content_img, style_weight, content_weight)
    input_param, optimizer = get_input_param_optimizer(input_img)
    #原来对每一张图像进行风格迁移，代表需要对每张图像都建立一个模型，都计算损失，都优化模型，都训练n次
    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_param.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_param)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.backward()
            for cl in content_losses:
                content_score += cl.backward()

            run[0] += 1
            
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.data.to('cpu').numpy(), content_score.data.to('cpu').numpy()))
                print()

            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    input_param.data.clamp_(0, 1)

    return model, style_losses, content_losses, input_param.data #返回优化后的网络模型损失和风格迁移结果

In [11]:
def get_cnn4():
    cnn = models.vgg19(pretrained=True).features
    # move it to the GPU if possible:
    if use_cuda:
        cnn = cnn.cuda()

    model_vgg=cnn[0:8]
    if use_cuda:
        model_vgg = model_vgg.cuda()
    return model_vgg

In [12]:
def image_unloader(tensor):
    image = tensor.clone().to(device)  # we clone the tensor to not do changes on it
    image = image.view(3, imsize, imsize)  # remove the fake batch dimension
    image = unloader(image)
    return image

In [13]:
# style_img = image_loader(LOAD_PATH).type(dtype) #.squeeze()
# #load content
# content_img = image_loader(CONTENT_PATH).type(dtype) #.squeeze()
# input_img = content_img.clone()

# model, style_losses, content_losses, output = run_style_transfer(cnn, content_img, style_img, input_img, 50)

In [14]:
LOAD_PATH = "./style"
SAVE_PATH = "./zero-watermark"
CONTENT_PATH = "content/l2.png"
if not os.path.exists(SAVE_PATH):
    os.mkdir(SAVE_PATH)  #创建

In [15]:
img_lst = os.listdir(LOAD_PATH)
img_lst.sort()

In [16]:
STEP = 100
i = 1
for img_name in img_lst:
    print("[%d/%d]" % (i, len(img_lst)))
    i = i + 1
    if os.path.splitext(img_name)[-1] != '.png':
        continue
    img_path = os.path.join(LOAD_PATH, img_name)
    style_img = image_loader(img_path).type(dtype).to(device)#.squeeze()
    #load content
    content_img = image_loader(CONTENT_PATH).type(dtype).to(device)#.squeeze()
    input_img = content_img.clone() #把内容图作为输入图，不是说好的白噪声么
#     print(content_img.shape)
    model, style_losses, content_losses, output = run_style_transfer(cnn, content_img, style_img, input_img, STEP)
    img = output.type(dtype).to(device)
    a = image_unloader(img)
    a.save(os.path.join(SAVE_PATH, img_name))

[1/9]
Building the style transfer model..
gram GramMatrix()
Optimizing..
run [50]:
Style Loss : 0.849238 Content Loss: 7.607591

run [100]:
Style Loss : 0.828870 Content Loss: 7.226240

[2/9]
Building the style transfer model..
gram GramMatrix()
Optimizing..
run [50]:
Style Loss : 2.299284 Content Loss: 5.733112

run [100]:
Style Loss : 1.196197 Content Loss: 5.552648

[3/9]
Building the style transfer model..
gram GramMatrix()
Optimizing..
run [50]:
Style Loss : 0.864887 Content Loss: 7.657245

run [100]:
Style Loss : 0.813435 Content Loss: 7.258070

[4/9]
Building the style transfer model..
gram GramMatrix()
Optimizing..
run [50]:
Style Loss : 1.803954 Content Loss: 5.322325

run [100]:
Style Loss : 0.717991 Content Loss: 5.112463

[5/9]
Building the style transfer model..
gram GramMatrix()
Optimizing..
run [50]:
Style Loss : 2.168500 Content Loss: 5.868726

run [100]:
Style Loss : 1.218965 Content Loss: 5.707106

[6/9]
Building the style transfer model..
gram GramMatrix()
Optimizing